# Practice Writing SQL Queries using Real Dataset with SPARK

#### Download Data from Kaggle:
FROM HERE(https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results)

List of the functions/operators used in this practice are:
- JOIN
- LEFT/RIGHT JOIN
- IN 
- DENSE_RANK
- PARTITION BY
- ...



In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

session_conf = (
    SparkConf()
    .setAppName("Practice SQL")
    .setMaster("local[1]")
)

spark = (
    SparkSession.builder
    .config(conf=session_conf)
    .getOrCreate()
)

In [3]:
spark

Data from athlete_events file. We load this data into a table named “OLYMPICS_HISTORY”.

In [163]:
athlete_eventsDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.option("quote", "\"")\
.option("escape", "\"")\
.load("mydata/dataset-sql-practice/athlete_events.csv")

In [164]:
athlete_eventsDataFrame.show(5)

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+-------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|     City|        Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+---------+-------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|Barcelona|   Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|   London|         Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|Antwerpen|     Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summer|    Paris|   Tug-Of-War|Tug-Of-War Men's ...| Gold|

In [165]:
athlete_eventsDataFrame.createOrReplaceTempView("OLYMPICS_HISTORY")

In [166]:
spark.sql("""
SELECT count(*)
From OLYMPICS_HISTORY
""").show()

+--------+
|count(1)|
+--------+
|  271116|
+--------+



Data from noc_regions file. We load this data into a table named “OLYMPICS_HISTORY_NOC_REGIONS”.

In [7]:
noc_regionsDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("mydata/dataset-sql-practice/noc_regions.csv")

In [8]:
noc_regionsDataFrame.show(5)

+---+-----------+--------------------+
|NOC|     region|               notes|
+---+-----------+--------------------+
|AFG|Afghanistan|                null|
|AHO|    Curacao|Netherlands Antilles|
|ALB|    Albania|                null|
|ALG|    Algeria|                null|
|AND|    Andorra|                null|
+---+-----------+--------------------+
only showing top 5 rows



In [9]:
noc_regionsDataFrame.createOrReplaceTempView("OLYMPICS_HISTORY_NOC_REGION")

In [10]:
spark.sql("show databases;").collect()

[Row(namespace='default')]

In [11]:
spark.sql("show tables in default;").collect()

[Row(namespace='', tableName='olympics_history', isTemporary=True),
 Row(namespace='', tableName='olympics_history_noc_region', isTemporary=True)]

In [12]:
noc_regionsDataFrame.show(1)

+---+-----------+-----+
|NOC|     region|notes|
+---+-----------+-----+
|AFG|Afghanistan| null|
+---+-----------+-----+
only showing top 1 row



In [13]:
athlete_eventsDataFrame.show(1)

+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
| ID|     Name|Sex|Age|Height|Weight| Team|NOC|      Games|Year|Season|     City|     Sport|               Event|Medal|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
|  1|A Dijiang|  M| 24|   180|    80|China|CHN|1992 Summer|1992|Summer|Barcelona|Basketball|Basketball Men's ...|   NA|
+---+---------+---+---+------+------+-----+---+-----------+----+------+---------+----------+--------------------+-----+
only showing top 1 row



### 1 How many olympics games have been held?

In [14]:
total_olympic_games = spark.sql("""
SELECT count(distinct games) as total_olympic_games
FROM olympics_history
""")

total_olympic_games.explain()
total_olympic_games.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(distinct games#25)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#113]
      +- HashAggregate(keys=[], functions=[partial_count(distinct games#25)])
         +- HashAggregate(keys=[games#25], functions=[])
            +- Exchange hashpartitioning(games#25, 200), ENSURE_REQUIREMENTS, [id=#109]
               +- HashAggregate(keys=[games#25], functions=[])
                  +- FileScan csv [Games#25] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/mydata/dataset-sql-practice/athlete_events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Games:string>


+-------------------+
|total_olympic_games|
+-------------------+
|                 80|
+-------------------+



### 2 List down all Olympics games held so far.

In [168]:
total_olympic_games = spark.sql("""
select distinct oh.year,oh.season,oh.city
from olympics_history as oh
order by year;
""")

total_olympic_games.explain()
total_olympic_games.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [year#2454 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(year#2454 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#8963]
      +- HashAggregate(keys=[year#2454, season#2455, city#2456], functions=[])
         +- Exchange hashpartitioning(year#2454, season#2455, city#2456, 200), ENSURE_REQUIREMENTS, [id=#8960]
            +- HashAggregate(keys=[year#2454, season#2455, city#2456], functions=[])
               +- FileScan csv [Year#2454,Season#2455,City#2456] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/mydata/dataset-sql-practice/athlete_events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Year:int,Season:string,City:string>


+----+------+--------------------+
|year|season|                city|
+----+------+--------------------+
|1896|Summer|              Athina|
|1900|Summer|               Paris|
|1904|Summer|           St. Louis

### 3 Mention the total no of nations who participated in each olympics game?

In [67]:
total_nb_nations_in_games = spark.sql("""
WITH nations_games AS
(
    SELECT distinct Games, region
    FROM olympics_history as a
        INNER JOIN OLYMPICS_HISTORY_NOC_REGION as b
        ON a.NOC=b.NOC
)
SELECT Games, count(1) as count
FROM nations_games
GROUP BY Games
ORDER BY Games
""").show()

+-----------+-----+
|      Games|count|
+-----------+-----+
|1896 Summer|   12|
|1900 Summer|   31|
|1904 Summer|   14|
|1906 Summer|   20|
|1908 Summer|   22|
|1912 Summer|   29|
|1920 Summer|   29|
|1924 Summer|   45|
|1924 Winter|   19|
|1928 Summer|   46|
|1928 Winter|   25|
|1932 Summer|   47|
|1932 Winter|   17|
|1936 Summer|   49|
|1936 Winter|   28|
|1948 Summer|   58|
|1948 Winter|   28|
|1952 Summer|   66|
|1952 Winter|   30|
|1956 Summer|   70|
+-----------+-----+
only showing top 20 rows



### 4 Which year saw the highest and lowest no of countries participating in olympics?

In [169]:
max_min_nb_countries_game = spark.sql("""
WITH nations_games AS
(
    SELECT year, count(distinct region) as count
    FROM olympics_history as a
        INNER JOIN OLYMPICS_HISTORY_NOC_REGION as b
        ON a.NOC=b.NOC
    GROUP BY year
),
max_year AS
(
    SELECT year
    FROM nations_games
    WHERE count=(SELECT max(count) FROM nations_games)
),
min_year AS
(
    SELECT year
    FROM nations_games
    WHERE count=(SELECT min(count) FROM nations_games)
)
SELECT max_year.year as max_year, min_year.year as min_year
FROM max_year join min_year
""")

# total_olympic_games.explain()
max_min_nb_countries_game.show()

+--------+--------+
|max_year|min_year|
+--------+--------+
|    2016|    1896|
+--------+--------+



## 5. Which nation has participated in all of the olympic games

In [68]:
spark.sql("""
WITH nations_games AS
(
    SELECT region, count(distinct Games) as count
    FROM olympics_history as a
        RIGHT JOIN OLYMPICS_HISTORY_NOC_REGION as b
        ON a.NOC=b.NOC
    GROUP BY region
)
SELECT region, count FROM nations_games WHERE count<(SELECT count(distinct year) from olympics_history) order by count desc
""").show()

+--------------+-----+
|        region|count|
+--------------+-----+
|            UK|   51|
|   Switzerland|   51|
|        France|   51|
|         Italy|   51|
|           USA|   50|
|       Austria|   50|
|        Sweden|   50|
|        Norway|   49|
|       Hungary|   49|
|        Canada|   49|
|     Australia|   48|
|       Finland|   48|
|Czech Republic|   48|
|        Greece|   47|
|       Belgium|   47|
|   Netherlands|   47|
|       Germany|   46|
|        Poland|   44|
|     Argentina|   42|
|         Spain|   42|
+--------------+-----+
only showing top 20 rows



## Identify the sport which was played in all summer olympics.

Logic: we want the sport which is played in summer and has count equal to the count of all unique summer games

In [103]:
spark.sql("""
WITH summer_olympics_sports AS
(
    SELECT distinct year, season, sport 
    FROM olympics_history 
    where season=="Summer"
    ORDER BY year
),
sport_count_game
(
SELECT sport, count(year,season) as count_game
FROM summer_olympics_sports
GROUP BY Sport
)
SELECT sport, count_game
FROM sport_count_game
WHERE count_game == (SELECT count(distinct year, season) from summer_olympics_sports)
""").show()


+----------+----------+
|     sport|count_game|
+----------+----------+
|Gymnastics|        29|
|  Swimming|        29|
| Athletics|        29|
|   Cycling|        29|
|   Fencing|        29|
+----------+----------+



## Which Sports were played only once in the olympics?

In [170]:
spark.sql("""
WITH sport_games_table AS
(
    SELECT distinct sport, games
    FROM olympics_history
),
count_games_table AS
(
    SELECT sport, count(games) as count_games
    FROM sport_games_table
    GROUP BY sport
)
SELECT count_games_table.sport, count_games, sport_games_table.games
FROM count_games_table
join sport_games_table on sport_games_table.sport=count_games_table.sport
WHERE count_games_table.count_games=1
""").show()

+-------------------+-----------+-----------+
|              sport|count_games|      games|
+-------------------+-----------+-----------+
|            Croquet|          1|1900 Summer|
|           Racquets|          1|1908 Summer|
|        Aeronautics|          1|1936 Summer|
|      Basque Pelota|          1|1900 Summer|
|       Rugby Sevens|          1|2016 Summer|
|              Roque|          1|1904 Summer|
|            Cricket|          1|1900 Summer|
|Military Ski Patrol|          1|1924 Winter|
|       Jeu De Paume|          1|1908 Summer|
|       Motorboating|          1|1908 Summer|
+-------------------+-----------+-----------+



## Fetch the total no of sports played in each olympic games.

In [134]:
spark.sql("""
SELECT games, count(distinct sport) as count_sport
FROM olympics_history
GROUP BY games
SORT BY count_sport desc
""").show()

+-----------+-----------+
|      games|count_sport|
+-----------+-----------+
|        USA|         40|
|2008 Summer|         34|
|2004 Summer|         34|
|2000 Summer|         34|
|2016 Summer|         34|
|2012 Summer|         32|
|1996 Summer|         31|
|1992 Summer|         29|
|1988 Summer|         27|
|1984 Summer|         25|
|1920 Summer|         25|
|1908 Summer|         24|
|1936 Summer|         24|
|1976 Summer|         23|
|1972 Summer|         23|
|1980 Summer|         23|
|1964 Summer|         21|
|1968 Summer|         20|
|1900 Summer|         20|
|1948 Summer|         20|
+-----------+-----------+
only showing top 20 rows



## Fetch details of the oldest athletes to win a gold medal.

In [154]:
spark.sql("""
WITH gold_medal_winner AS
(
    SELECT *
    FROM olympics_history
    WHERE medal=="Gold" and age!="NA"
    SORT BY age desc
)
SELECT gold_medal_winner.* 
FROM gold_medal_winner
Where age == (SELECT max(age) from gold_medal_winner)

""").show()

+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+--------------------+-----+
|    ID|             Name|Sex|Age|Height|Weight|         Team|NOC|      Games|Year|Season|     City|   Sport|               Event|Medal|
+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+--------------------+-----+
| 53238|  Charles Jacobus|  M| 64|    NA|    NA|United States|USA|1904 Summer|1904|Summer|St. Louis|   Roque| Roque Men's Singles| Gold|
|117046|Oscar Gomer Swahn|  M| 64|    NA|    NA|       Sweden|SWE|1912 Summer|1912|Summer|Stockholm|Shooting|Shooting Men's Ru...| Gold|
+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+--------------------+-----+



## Find the Ratio of male and female athletes participated in all olympic games.

In [155]:
spark.sql("""
SELECT sum(case when `SEX` = 'M' then 1 else 0 end)/count(*) as male_ratio,
       sum(case when `SEX` = 'F' then 1 else 0 end)/count(*) as female_ratio
FROM olympics_history
""").show()

+------------------+------------------+
|        male_ratio|      female_ratio|
+------------------+------------------+
|0.7232549904837782|0.2742626772304106|
+------------------+------------------+



### Fetch the top 5 athletes who have won the most gold medals.

In [119]:
spark.sql("""
WITH athlete_medals_count_table AS
(
    SELECT name, team, count(medal) as athlete_medals_count
    FROM olympics_history
    WHERE medal="Gold"
    GROUP BY name, team
    Order by athlete_medals_count desc
),
top_5_won_medal_table AS
(
    SELECT distinct athlete_medals_count
    FROM athlete_medals_count_table
    ORDER by athlete_medals_count DESC
    LIMIT 5
)
SELECT * 
FROM athlete_medals_count_table
WHERE athlete_medals_count IN(
        SELECT athlete_medals_count 
        FROM top_5_won_medal_table 
        )
ORDER BY athlete_medals_count desc
""").show()



+--------------------+--------------+--------------------+
|                name|          team|athlete_medals_count|
+--------------------+--------------+--------------------+
|Michael Fred Phel...| United States|                  23|
|"Raymond Clarence...| United States|                  10|
|   Mark Andrew Spitz| United States|                   9|
|"Frederick Carlto...| United States|                   9|
|Larysa Semenivna ...|  Soviet Union|                   9|
|Paavo Johannes Nurmi|       Finland|                   9|
|  Usain St. Leo Bolt|       Jamaica|                   8|
|"Matthew Nicholas...| United States|                   8|
|"Jennifer Elisabe...| United States|                   8|
|          Sawao Kato|         Japan|                   8|
| Ole Einar Bjrndalen|        Norway|                   8|
|"Donald Arthur ""...| United States|                   7|
|Vra slavsk (-Odlo...|Czechoslovakia|                   7|
|Viktor Ivanovych ...|  Soviet Union|                   

### Fetch the top 5 athletes who have won the most medals (gold/silver/bronze).
Plus: create a rank

In [148]:
#  with ranking
spark.sql("""
WITH athlete_medals_count_table AS
(
    SELECT ID, name, team, count(medal) as athlete_medals_count
    FROM olympics_history
    WHERE medal In("Silver", "Gold", "Bronze")
    GROUP BY ID,name, team
    Order by athlete_medals_count desc
),
top_5_won_medal_table AS
(
    SELECT distinct athlete_medals_count
    FROM athlete_medals_count_table
    ORDER by athlete_medals_count DESC
    LIMIT 5
),
top_5_athletes_table AS
(
    SELECT * 
    FROM athlete_medals_count_table
    WHERE athlete_medals_count IN(
            SELECT athlete_medals_count 
            FROM top_5_won_medal_table 
            )
)
SELECT t.ID, t.name, t.team, athlete_medals_count, DENSE_RANK() OVER(ORDER BY t.athlete_medals_count DESC) as Rank
FROM top_5_athletes_table t
""").show()



+------+--------------------+-------------+--------------------+----+
|    ID|                name|         team|athlete_medals_count|Rank|
+------+--------------------+-------------+--------------------+----+
| 94406|Michael Fred Phel...|United States|                  28|   1|
| 67046|Larysa Semenivna ...| Soviet Union|                  18|   2|
|  4198|Nikolay Yefimovic...| Soviet Union|                  15|   3|
| 11951| Ole Einar Bjrndalen|       Norway|                  13|   4|
| 74420| Edoardo Mangiarotti|        Italy|                  13|   4|
|109161|Borys Anfiyanovyc...| Soviet Union|                  13|   4|
| 89187|         Takashi Ono|        Japan|                  13|   4|
| 23426|Natalie Anne Coug...|United States|                  12|   5|
| 85286|Aleksey Yuryevich...|       Russia|                  12|   5|
| 87390|Paavo Johannes Nurmi|      Finland|                  12|   5|
| 57998|          Sawao Kato|        Japan|                  12|   5|
|119922|"Jennifer El

In [149]:
# using row_number
spark.sql("""
WITH athlete_medals_count_table AS
(
    SELECT ID, name, team, count(medal) as athlete_medals_count
    FROM olympics_history
    WHERE medal In("Silver", "Gold", "Bronze")
    GROUP BY ID,name, team
    Order by athlete_medals_count desc
),
top_5_won_medal_table AS
(
    SELECT distinct athlete_medals_count
    FROM athlete_medals_count_table
    ORDER by athlete_medals_count DESC
    LIMIT 5
),
top_5_athletes_table AS
(
    SELECT * 
    FROM athlete_medals_count_table
    WHERE athlete_medals_count IN(
            SELECT athlete_medals_count 
            FROM top_5_won_medal_table 
            )
)
SELECT t.ID, t.name, t.team, athlete_medals_count, ROW_NUMBER() OVER(ORDER BY t.athlete_medals_count DESC) as Rank
FROM top_5_athletes_table t
""").show()



+------+--------------------+-------------+--------------------+----+
|    ID|                name|         team|athlete_medals_count|Rank|
+------+--------------------+-------------+--------------------+----+
| 94406|Michael Fred Phel...|United States|                  28|   1|
| 67046|Larysa Semenivna ...| Soviet Union|                  18|   2|
|  4198|Nikolay Yefimovic...| Soviet Union|                  15|   3|
| 11951| Ole Einar Bjrndalen|       Norway|                  13|   4|
| 74420| Edoardo Mangiarotti|        Italy|                  13|   5|
|109161|Borys Anfiyanovyc...| Soviet Union|                  13|   6|
| 89187|         Takashi Ono|        Japan|                  13|   7|
| 23426|Natalie Anne Coug...|United States|                  12|   8|
| 85286|Aleksey Yuryevich...|       Russia|                  12|   9|
| 87390|Paavo Johannes Nurmi|      Finland|                  12|  10|
| 57998|          Sawao Kato|        Japan|                  12|  11|
|119922|"Jennifer El

### Fetch the top 5 most successful countries in olympics. Success is defined by no of medals won.

In [150]:
spark.sql("""
WITH winning_noc_table AS
(
    SELECT NOC, count(medal) as noc_medals_count
    FROM olympics_history
    WHERE medal != "NA"
    GROUP BY NOC
),
top_5_winners_table AS
(
    SELECT distinct NOC, noc_medals_count
    FROM winning_noc_table
    ORDER BY noc_medals_count desc
    LIMIT 5
)
SELECT Region, top_5_winners_table.NOC, noc_medals_count, DENSE_RANK() OVER(ORDER BY noc_medals_count DESC) as Rank
FROM OLYMPICS_HISTORY_NOC_REGION
RIGHT JOIN top_5_winners_table on top_5_winners_table.NOC==OLYMPICS_HISTORY_NOC_REGION.NOC

""").show()



+-------+---+----------------+----+
| Region|NOC|noc_medals_count|Rank|
+-------+---+----------------+----+
|    USA|USA|            5437|   1|
| Russia|URS|            2502|   2|
|Germany|GER|            2162|   3|
|     UK|GBR|            2066|   4|
| France|FRA|            1777|   5|
+-------+---+----------------+----+



### List down total gold, silver and bronze medals won by each country.

In [211]:
spark.sql("""
WITH noc_medals AS
(
    SELECT NOC, medal, count(medal) as medal_count
    FROM olympics_history
    Where medal!="NA"
    GROUP BY NOC, medal
    ORDER BY NOC, medal
)
SELECT noc_medals.NOC, OLYMPICS_HISTORY_NOC_REGION.region as region, medal, medal_count, 
RANK() OVER(PARTITION BY region ORDER BY medal_count DESC) Rank
FROM noc_medals
left JOIN OLYMPICS_HISTORY_NOC_REGION on OLYMPICS_HISTORY_NOC_REGION.NOC==noc_medals.NOC

""").show()



+---+-----------+------+-----------+----+
|NOC|     region| medal|medal_count|Rank|
+---+-----------+------+-----------+----+
|SGP|       null|Bronze|          4|   1|
|SGP|       null|Silver|          4|   1|
|SGP|       null|  Gold|          1|   3|
|AFG|Afghanistan|Bronze|          2|   1|
|ALG|    Algeria|Bronze|          8|   1|
|ALG|    Algeria|  Gold|          5|   2|
|ALG|    Algeria|Silver|          4|   3|
|ARG|  Argentina|Silver|         92|   1|
|ARG|  Argentina|Bronze|         91|   2|
|ARG|  Argentina|  Gold|         91|   2|
|ARM|    Armenia|Bronze|          9|   1|
|ARM|    Armenia|Silver|          5|   2|
|ARM|    Armenia|  Gold|          2|   3|
|AUS|  Australia|Bronze|        517|   1|
|AUS|  Australia|Silver|        455|   2|
|AUS|  Australia|  Gold|        348|   3|
|ANZ|  Australia|  Gold|         20|   4|
|ANZ|  Australia|Bronze|          5|   5|
|ANZ|  Australia|Silver|          4|   6|
|AUT|    Austria|Silver|        186|   1|
+---+-----------+------+----------

In [223]:
spark.sql("""
WITH noc_medals AS
(
    SELECT NOC, medal, count(medal) as medal_count
    FROM olympics_history
    Where medal!="NA"
    GROUP BY NOC, medal
    ORDER BY NOC, medal
),
region_medal AS
(
    SELECT noc_medals.NOC, OLYMPICS_HISTORY_NOC_REGION.region, medal, medal_count
    FROM noc_medals
    left JOIN OLYMPICS_HISTORY_NOC_REGION on OLYMPICS_HISTORY_NOC_REGION.NOC==noc_medals.NOC
),
region_medal_count AS
(
    SELECT region, medal, sum(medal_count) as medal_count 
    FROM region_medal
    GROUP BY region, medal
    ORDER BY region, medal
)
SELECT region, medal, medal_count, 
RANK() OVER(PARTITION BY region ORDER BY medal_count DESC) Rank
FROM region_medal_count
""").show()


+-----------+------+-----------+----+
|     region| medal|medal_count|Rank|
+-----------+------+-----------+----+
|       null|Bronze|          4|   1|
|       null|Silver|          4|   1|
|       null|  Gold|          1|   3|
|Afghanistan|Bronze|          2|   1|
|    Algeria|Bronze|          8|   1|
|    Algeria|  Gold|          5|   2|
|    Algeria|Silver|          4|   3|
|  Argentina|Silver|         92|   1|
|  Argentina|Bronze|         91|   2|
|  Argentina|  Gold|         91|   2|
|    Armenia|Bronze|          9|   1|
|    Armenia|Silver|          5|   2|
|    Armenia|  Gold|          2|   3|
|  Australia|Bronze|        522|   1|
|  Australia|Silver|        459|   2|
|  Australia|  Gold|        368|   3|
|    Austria|Silver|        186|   1|
|    Austria|Bronze|        156|   2|
|    Austria|  Gold|        108|   3|
| Azerbaijan|Bronze|         25|   1|
+-----------+------+-----------+----+
only showing top 20 rows



### List down total gold, silver and bronze medals won by each country corresponding to each olympic games.

In [233]:
spark.sql("""
WITH noc_medals AS
(
    SELECT NOC, games, medal, count(medal) as medal_count
    FROM olympics_history
    Where medal!="NA"
    GROUP BY NOC, games, medal
    ORDER BY NOC, games, medal
),
region_medal AS
(
    SELECT noc_medals.NOC, OLYMPICS_HISTORY_NOC_REGION.region, games, medal, medal_count
    FROM noc_medals
    left JOIN OLYMPICS_HISTORY_NOC_REGION on OLYMPICS_HISTORY_NOC_REGION.NOC==noc_medals.NOC
),
region_medal_count AS
(
    SELECT region, games, medal, sum(medal_count) as medal_count 
    FROM region_medal
    GROUP BY region, games, medal
    ORDER BY region, games, medal
)
SELECT region, games, medal, medal_count, 
RANK() OVER(PARTITION BY region ORDER BY medal_count DESC) Rank
FROM region_medal_count
""").show()


+-----------+-----------+------+-----------+----+
|     region|      games| medal|medal_count|Rank|
+-----------+-----------+------+-----------+----+
|       null|2012 Summer|Bronze|          4|   1|
|       null|2008 Summer|Silver|          3|   2|
|       null|1960 Summer|Silver|          1|   3|
|       null|2016 Summer|  Gold|          1|   3|
|Afghanistan|2008 Summer|Bronze|          1|   1|
|Afghanistan|2012 Summer|Bronze|          1|   1|
|    Algeria|2000 Summer|Bronze|          3|   1|
|    Algeria|1984 Summer|Bronze|          2|   2|
|    Algeria|1996 Summer|  Gold|          2|   2|
|    Algeria|2016 Summer|Silver|          2|   2|
|    Algeria|1992 Summer|Bronze|          1|   5|
|    Algeria|1992 Summer|  Gold|          1|   5|
|    Algeria|1996 Summer|Bronze|          1|   5|
|    Algeria|2000 Summer|  Gold|          1|   5|
|    Algeria|2000 Summer|Silver|          1|   5|
|    Algeria|2008 Summer|Bronze|          1|   5|
|    Algeria|2008 Summer|Silver|          1|   5|


###  Identify which country won the most gold, most silver and most bronze medals in each olympic games.

2 ways to transpose the data

In [279]:
# way 1
spark.sql("""
WITH noc_medals AS
(
    SELECT NOC, games, medal, count(medal) as medal_count
    FROM olympics_history
    Where medal!="NA"
    GROUP BY NOC, games, medal
    ORDER BY NOC, games, medal
),
region_medal AS
(
    SELECT noc_medals.NOC, OLYMPICS_HISTORY_NOC_REGION.region, games, medal, medal_count
    FROM noc_medals
    left JOIN OLYMPICS_HISTORY_NOC_REGION on OLYMPICS_HISTORY_NOC_REGION.NOC==noc_medals.NOC
),
region_medal_count AS
(
    SELECT region, games, medal, sum(medal_count) as medal_count 
    FROM region_medal
    GROUP BY region, games, medal
    ORDER BY region, games, medal
),
country_won_most_medal_per_game AS
(

    SELECT games, medal, region, medal_count, 
    RANK() OVER(PARTITION BY games, medal ORDER BY medal_count DESC) Rank
    FROM region_medal_count
),
answer AS
(
    SELECT games, medal, region, medal_count
    FROM country_won_most_medal_per_game
    WHERE Rank==1
)
SELECT games,
        Max(CASE WHEN medal = 'Gold' THEN Concat(region, ' - ', medal_count) END) AS Gold,
        Max(CASE WHEN medal = 'Silver' THEN Concat(region, ' - ', medal_count) END) AS Silver,
        Max(CASE WHEN medal = 'Bronze' THEN Concat(region, ' - ', medal_count) END) AS Bronze
FROM answer
GROUP BY games
""").show()


+-----------+------------+-------------------+----------------+
|      games|        Gold|             Silver|          Bronze|
+-----------+------------+-------------------+----------------+
|1896 Summer|Germany - 25|        Greece - 18|     Greece - 20|
|1900 Summer|     UK - 59|       France - 101|     France - 82|
|1904 Summer|   USA - 128|          USA - 141|       USA - 125|
|1906 Summer| Greece - 24|        Greece - 48|     Greece - 30|
|1908 Summer|    UK - 147|           UK - 131|         UK - 90|
|1912 Summer|Sweden - 103|            UK - 64|         UK - 59|
|1920 Summer|   USA - 111|        France - 71|    Belgium - 66|
|1924 Summer|    USA - 97|        France - 51|        USA - 49|
|1924 Winter|     UK - 16|           USA - 10|         UK - 11|
|1928 Summer|    USA - 47|   Netherlands - 29|    Germany - 41|
|1928 Winter| Canada - 12|        Sweden - 13|Switzerland - 12|
|1932 Summer|    USA - 81|           USA - 47|        USA - 61|
|1932 Winter| Canada - 14|           USA

In [ ]:
# way 2
spark.sql("""
WITH noc_medals AS
(
    SELECT NOC, games, medal, count(medal) as medal_count
    FROM olympics_history
    Where medal!="NA"
    GROUP BY NOC, games, medal
    ORDER BY NOC, games, medal
),
region_medal AS
(
    SELECT noc_medals.NOC, OLYMPICS_HISTORY_NOC_REGION.region, games, medal, medal_count
    FROM noc_medals
    left JOIN OLYMPICS_HISTORY_NOC_REGION on OLYMPICS_HISTORY_NOC_REGION.NOC==noc_medals.NOC
),
region_medal_count AS
(
    SELECT region, games, medal, sum(medal_count) as medal_count 
    FROM region_medal
    GROUP BY region, games, medal
    ORDER BY region, games, medal
),
country_won_most_medal_per_game AS
(

    SELECT games, medal, region, medal_count, 
    RANK() OVER(PARTITION BY games, medal ORDER BY medal_count DESC) Rank
    FROM region_medal_count
),
answer_gold AS
(
    SELECT games, medal, region, medal_count
    FROM country_won_most_medal_per_game
    WHERE Rank==1 and medal=='Gold'
),
answer_silver AS
(
    SELECT games, medal, region, medal_count
    FROM country_won_most_medal_per_game
    WHERE Rank==1 and medal=='Silver'
),
answer_bronze AS
(
    SELECT games, medal, region, medal_count
    FROM country_won_most_medal_per_game
    WHERE Rank==1 and medal=='Bronze'
)
SELECT answer_gold.games, 
    CONCAT(answer_gold.region, ' - ', answer_gold.medal_count) as Gold,
    CONCAT(answer_silver.region, ' - ', answer_silver.medal_count) as Silver,
    CONCAT(answer_bronze.region, ' - ', answer_bronze.medal_count) as Bronze
FROM answer_gold
JOIN answer_silver 
JOIN answer_bronze
on answer_gold.games=answer_silver.games and answer_gold.games=answer_bronze.games
""").show()


### Identify which country won the most gold, most silver, most bronze medals and the most medals in each olympic games.

### Which countries have never won gold medal but have won silver/bronze medals?

### Break down all olympic games where India won medal for Hockey and how many medals in each olympic games